In [ ]:
import rospy

import math
import tf2_ros
from geometry_msgs.msg import Twist
from turtlesim.srv import Spawn

In [ ]:
rospy.init_node('tf2_turtle_listener')

In [ ]:
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [ ]:
turtle_name = 'turtle2'
rospy.wait_for_service('spawn')
spawner = rospy.ServiceProxy('spawn', Spawn)
spawner(4, 2, 0, turtle_name)

In [ ]:
turtle_vel = rospy.Publisher('%s/cmd_vel' % turtle_name, \
                             Twist, queue_size=1)
rate = rospy.Rate(10.0)

In [ ]:
while not rospy.is_shutdown():
    try:
        trans = tfBuffer.lookup_transform(turtle_name, \
                                          'turtle1', rospy.Time())
        tr = trans.transform.translation
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, \
            tf2_ros.ExtrapolationException):
        rate.sleep()
        continue

    msg = Twist()
    msg.angular.z = 4 * math.atan2(tr.y, tr.x)
    msg.linear.x = 0.5 * math.sqrt(tr.x ** 2 + tr.y ** 2)

    turtle_vel.publish(msg)

    rate.sleep()